#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
!pip install pymongo

In [2]:
import pandas as pd
import pymongo

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [6]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [7]:
df_ks.shape

(150000, 14)

### Importer les données

In [8]:
df_ks = df_ks.to_json(orient="records")

In [9]:
import json

df_ks = json.loads(df_ks)

In [10]:
collection.insert_many(df_ks)

## Question 1  

In [11]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('61f77537cf6fb2fbce0210b5'),
  'ID': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': '2016-05-10 15:53:56',
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('61f77537cf6fb2fbce01bf93'),
  'ID': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': '2014-10-23 20:06:37',
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': None,
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('61f77537cf6fb2fbce019743'),
  'ID': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'm

## Question 2

In [12]:
count_cur = collection.find({"state":"successful"}).count()
count_cur

/tmp/ipykernel_15/1067602021.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count_cur = collection.find({"state":"successful"}).count()


53040

## Question 3

In [13]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "Count" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'DIY Electronics', 'Count': 358},
 {'_id': 'Plays', 'Count': 539},
 {'_id': 'Printing', 'Count': 83},
 {'_id': 'Space Exploration', 'Count': 137},
 {'_id': 'Hip-Hop', 'Count': 1555},
 {'_id': 'Illustration', 'Count': 1263},
 {'_id': 'Crafts', 'Count': 1834},
 {'_id': 'Couture', 'Count': 108},
 {'_id': 'Horror', 'Count': 525},
 {'_id': 'Installations', 'Count': 178},
 {'_id': 'World Music', 'Count': 850},
 {'_id': 'Ready-to-wear', 'Count': 332},
 {'_id': 'Blues', 'Count': 113},
 {'_id': 'Digital Art', 'Count': 524},
 {'_id': 'Musical', 'Count': 367},
 {'_id': 'Workshops', 'Count': 59},
 {'_id': 'Bacon', 'Count': 78},
 {'_id': 'Pet Fashion', 'Count': 51},
 {'_id': 'Web', 'Count': 2017},
 {'_id': 'Art', 'Count': 3358},
 {'_id': 'Farms', 'Count': 482},
 {'_id': 'Video Art', 'Count': 65},
 {'_id': 'Candles', 'Count': 168},
 {'_id': 'Shorts', 'Count': 4857},
 {'_id': 'Gadgets', 'Count': 1210},
 {'_id': 'Fine Art', 'Count': 291},
 {'_id': 'Places', 'Count': 283},
 {'_id': 'Camera Equ

## Question 4

In [14]:
count_cur = collection.find({"$and":[{"country":"FR"}, {"launched":{"$lt":"2016-01-01"}}]}).count()
count_cur

/tmp/ipykernel_15/1721618137.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count_cur = collection.find({"$and":[{"country":"FR"}, {"launched":{"$lt":"2016-01-01"}}]}).count()


330

## Question 5

In [15]:
cur = collection.find({"pledged":{"$gt":200000}})
next(cur)

{'_id': ObjectId('61f77536cf6fb2fbce003b90'),
 'ID': 217543389,
 'name': 'The uKeg Pressurized Growler for Fresh Beer',
 'category': 'Drinks',
 'main_category': 'Food',
 'currency': 'USD',
 'deadline': '2014-12-08',
 'goal': 75000.0,
 'launched': '2014-10-15 06:34:48',
 'pledged': 1559525.68,
 'state': 'successful',
 'backers': 10293,
 'country': 'US',
 'usd pledged': 1559525.68,
 'usd_pledged_real': 1559525.68}

## Question 6 

In [16]:
cur = collection.find({"name": {"$regex" : 'Sport'}})
next(cur)

{'_id': ObjectId('61f77536cf6fb2fbce003f60'),
 'ID': 802281658,
 'name': 'Sportswear range',
 'category': 'Apparel',
 'main_category': 'Fashion',
 'currency': 'AUD',
 'deadline': '2014-08-23',
 'goal': 25000.0,
 'launched': '2014-07-24 05:14:52',
 'pledged': 20.0,
 'state': 'failed',
 'backers': 1,
 'country': 'AU',
 'usd pledged': 18.7569048,
 'usd_pledged_real': 18.6758801008}